In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

import kagglehub

# Download latest version
path = kagglehub.dataset_download("quynhlecl/lung-cancer-x-ray")
print("Path to dataset files:", path)



100%|██████████| 2.28G/2.28G [00:09<00:00, 249MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/quynhlecl/lung-cancer-x-ray/versions/1


In [ ]:
# Set the root dataset path
root_dataset_path = "/root/.cache/kagglehub/datasets/quynhlecl/lung-cancer-x-ray/versions/1/chest_xray_lung/"

train_dataset_path = os.path.join(root_dataset_path, "train")
test_dataset_path = os.path.join(root_dataset_path, "test")
val_dataset_path = os.path.join(root_dataset_path, "val")


In [ ]:
# Function to explore the dataset
def explore_dataset(dataset_path):
    class_counts = {}
    total_images = 0

    for root, dirs, files in os.walk(dataset_path):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            num_files = len(os.listdir(folder_path))
            class_counts[folder] = num_files
            total_images += num_files

    print("Dataset Overview:")
    print(f"Total Classes: {len(class_counts)}")
    print(f"Total Images: {total_images}")
    print("Class Distribution:")
    for cls, count in class_counts.items():
        print(f"  {cls}: {count}")

    return class_counts

class_counts = explore_dataset(root_dataset_path)



Dataset Overview:
Total Classes: 6
Total Images: 11727
Class Distribution:
  test: 2
  val: 2
  train: 2
  chest_xray: 3
  NORMAL: 1341
  Cancer: 3875


In [ ]:
# Calculate class weights for imbalanced data
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(list(class_counts.keys())),
    y=np.concatenate([[k] * v for k, v in class_counts.items()])
)
class_weights = dict(enumerate(class_weights))



In [ ]:
# Data Augmentation
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data Generators
train_generator = train_gen.flow_from_directory(
    train_dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True
)
val_generator = val_test_gen.flow_from_directory(
    val_dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)
test_generator = val_test_gen.flow_from_directory(
    test_dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)



Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# Model Definition using ResNet50
def create_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(train_generator.num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model()
model.summary()



94765736/94765736 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 23850242 (90.98 MB)
Trainable params: 26253

In [ ]:
from sklearn.model_selection import KFold

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_resnet.h5', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)


def extract_data_and_labels(generator):
    data = []
    labels = []
    for _ in range(len(generator)):
        images, batch_labels = generator.next()
        data.append(images)
        labels.append(batch_labels)
    return np.vstack(data), np.concatenate(labels)

# Extract data and labels
train_data, train_labels = extract_data_and_labels(train_generator)

# Initialize KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform K-Fold Cross-Validation
fold_no = 1
for train_idx, val_idx in kfold.split(train_data, train_labels):
    print(f"Training Fold {fold_no}...")

    # Create data generators for the current fold
    train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow(
        train_data[train_idx], train_labels[train_idx], batch_size=32
    )
    val_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow(
        train_data[val_idx], train_labels[val_idx], batch_size=32
    )

    # Create a new model instance
    model = create_model()

    # Train the model
    history = model.fit(
        train_data_gen,
        epochs=10,
        validation_data=val_data_gen,
        callbacks=[early_stopping, model_checkpoint]
    )

    # Evaluate the model
    val_loss, val_acc = model.evaluate(val_data_gen)
    print(f"Fold {fold_no} - Validation Accuracy: {val_acc:.2f}")

    fold_no += 1


Training Fold 1...
Epoch 1/10
131/131 [==============================] - ETA: 0s - loss: 0.2530 - accuracy: 0.9010

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


131/131 [==============================] - 898s 7s/step - loss: 0.2530 - accuracy: 0.9010 - val_loss: 0.2419 - val_accuracy: 0.8956
Epoch 2/10
131/131 [==============================] - 897s 7s/step - loss: 0.1535 - accuracy: 0.9382 - val_loss: 0.1325 - val_accuracy: 0.9492
Epoch 3/10
131/131 [==============================] - 878s 7s/step - loss: 0.1240 - accuracy: 0.9506 - val_loss: 0.1210 - val_accuracy: 0.9569
Epoch 4/10
131/131 [==============================] - 882s 7s/step - loss: 0.0984 - accuracy: 0.9619 - val_loss: 0.1298 - val_accuracy: 0.9473
Epoch 5/10
131/131 [==============================] - 876s 7s/step - loss: 0.0999 - accuracy: 0.9609 - val_loss: 0.1057 - val_accuracy: 0.9636
Epoch 6/10
131/131 [==============================] - 876s 7s/step - loss: 0.0963 - accuracy: 0.9640 - val_loss: 0.1200 - val_accuracy: 0.9550
Epoch 7/10
131/131 [==============================] - 876s 7s/step - loss: 0.0766 - accuracy: 0.9703 - val_loss: 0.1350 - val_accuracy: 0.9502
Epoch 8/10

In [ ]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

# Classification Report and Metrics
y_pred = np.argmax(model.predict(test_generator), axis=1)
y_true = test_generator.classes
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

# AUC-ROC Score
auc = roc_auc_score(y_true, y_pred)
print(f"AUC-ROC: {auc:.2f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()



NameError: name 'model' is not defined

In [ ]:
# Visualization of Training Histories
for idx, history in enumerate(histories):
    plt.plot(history.history['accuracy'], label=f'Fold {idx+1} Train Accuracy')
    plt.plot(history.history['val_accuracy'], label=f'Fold {idx+1} Validation Accuracy')

plt.title('Cross-Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
